In [2]:
%matplotlib inline
import pandas as pd
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
from pandas import *
import csv
from datetime import datetime
from dateutil import relativedelta
import argparse
import numpy as np
from itertools import chain
import math
import copy
pd.set_option('display.max_columns', 45)
import re
from dateutil.relativedelta import *

In [3]:
def partitioning(dataset, start_date, end_date): # choose what start and end date you want for you dataframe 
     principio =   int(dataset[dataset['date']== start_date].index.values[0])
    
     fin = int(dataset[dataset['date']== end_date].index.values[0])+1
     
     dataset = dataset[principio:fin].reset_index(drop=True)
     
     dataset['Timestamp'] = pd.DatetimeIndex(dataset['date'])#prophet readable datetime format

     #      dataset = dataset.rename(columns={'DATE_TIME': 'ds','DEMAND': 'y'})
    
     dataset['week#'] = dataset['date'].dt.week
     
     dataset['day#'] = dataset['date'].dt.day
     
     dataset['dayofweek#'] = dataset['date'].dt.dayofweek
     
     dataset['dayofyear#'] = dataset['date'].dt.dayofyear
    
     dataset['hour#'] = dataset['date'].dt.hour
     
    
     return dataset

In [4]:
df = pd.read_csv(".\static\data\monterrey_Total_P.csv")
df['date'] = pd.to_datetime(df.Timestamp, dayfirst=True)
part_df = partitioning(df,df.date[0],df.date[len(df.date)-1])
del part_df['Timestamp']
del part_df['Unnamed: 0']
part_df

,Total,date,week#,day#,dayofweek#,dayofyear#,hour#
0,577.019897,2020-11-30 00:00:00,49,30,0,335,0
1,564.285583,2020-11-30 01:00:00,49,30,0,335,1
2,552.912170,2020-11-30 02:00:00,49,30,0,335,2
3,561.570068,2020-11-30 03:00:00,49,30,0,335,3
4,568.570007,2020-11-30 04:00:00,49,30,0,335,4
5,572.646545,2020-11-30 05:00:00,49,30,0,335,5
6,572.796509,2020-11-30 06:00:00,49,30,0,335,6
7,569.288208,2020-11-30 07:00:00,49,30,0,335,7
8,559.199341,2020-11-30 08:00:00,49,30,0,335,8
9,550.328064,2020-11-30 09:00:00,49,30,0,335,9


In [7]:
# part_df.iloc[part_df.groupby('dayofyear#')['MONTERREY']]#.agg(pd.Series.idxmax)]#\
# #                             .sort_values(by=['dayofyear#']).reset_index(drop = True)
# part_df.iloc[part_df.groupby('dayofyear#')['MONTERREY'].agg(pd.Series.nlargest)]
# part_df.groupby('dayofyear#')['MONTERREY'].apply(pd.Series.nlargest)

data = part_df.groupby('dayofyear#').apply(lambda x: x.sort_values('Total',ascending=False)).reset_index(drop = True)

index_list = []

for i in data.index:
    if i%24 in [0,1,2,3]:# 24 will change depending on the values slected
        index_list.append(i)

datap2 = data.ix[index_list].reset_index(drop = True)
# datap2

C:\Users\goluk\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [8]:
price = pd.DataFrame(np.reshape(list(datap2.Total),(7,4)),columns=['top1_hour_price','top2_hour_price',\
                                                               'top3_hour_price','top4_hour_price'])
date = pd.DataFrame(np.reshape(list(datap2.date),(7,4)),columns=['top1_hour_date','top2_hour_date',\
                                                               'top3_hour_date','top4_hour_date'])
final_dataset = pd.concat([date, price], axis = 1)
final_dataset = final_dataset[['top1_hour_date', 'top1_hour_price','top2_hour_date', 'top2_hour_price',\
                               'top3_hour_date', 'top3_hour_price','top4_hour_date', 'top4_hour_price']]
final_dataset

,top1_hour_date,top1_hour_price,top2_hour_date,top2_hour_price,top3_hour_date,top3_hour_price,top4_hour_date,top4_hour_price
0,2020-11-30 00:00:00,577.019897,2020-11-30 06:00:00,572.796509,2020-11-30 05:00:00,572.646545,2020-11-30 07:00:00,569.288208
1,2020-12-01 06:00:00,549.327698,2020-12-01 05:00:00,549.089172,2020-12-01 07:00:00,546.466553,2020-12-01 08:00:00,542.126404
2,2020-12-02 06:00:00,537.992371,2020-12-02 07:00:00,537.102051,2020-12-02 10:00:00,537.056946,2020-12-02 05:00:00,536.524231
3,2020-12-03 11:00:00,600.445190,2020-12-03 12:00:00,599.379333,2020-12-03 10:00:00,594.976196,2020-12-03 13:00:00,592.679932
4,2020-12-04 09:00:00,756.442993,2020-12-04 10:00:00,755.974670,2020-12-04 11:00:00,748.604675,2020-12-04 08:00:00,748.138794
5,2020-12-05 07:00:00,611.794800,2020-12-05 06:00:00,610.978027,2020-12-05 08:00:00,609.392761,2020-12-05 05:00:00,602.383423
6,2020-12-06 16:00:00,531.785767,2020-12-06 17:00:00,527.544556,2020-12-06 15:00:00,521.560425,2020-12-06 18:00:00,510.309509


In [18]:
weekly_avg_rec = part_df.groupby('dayofyear#').mean().sort_values(by ='Total').reset_index(drop = True)
weekly_dict = {0.0:'Monday',1.0:'Tuesday',2.0:'Wednesday',3.0:'Thursday',4.0:'Friday',5.0:'Saturday',6.0:'Sunday'}

weekly_list = [weekly_dict[weekly_avg_rec['dayofweek#'][i]] for i in range(len(weekly_dict))]
weekly_avg_rec.insert(5, "Week_day", weekly_list, True)
weekly_avg_rec['Avg_price'] = weekly_avg_rec['Total']
weekly_avg_rec = weekly_avg_rec[['Week_day','Avg_price']]
weekly_avg_rec = weekly_avg_rec.T 
weekly_avg_rec.columns = ['Daily_Average_Price_1','Daily_Average_Price_2','Daily_Average_Price_3',\
                          'Daily_Average_Price_4','Daily_Average_Price_5','Daily_Average_Price_6',\
                          'Daily_Average_Price_7']
weekly_avg_rec = weekly_avg_rec.round(2)
weekly_avg_rec

,Daily_Average_Price_1,Daily_Average_Price_2,Daily_Average_Price_3,Daily_Average_Price_4,Daily_Average_Price_5,Daily_Average_Price_6,Daily_Average_Price_7
Week_day,Sunday,Tuesday,Wednesday,Monday,Saturday,Thursday,Friday
Avg_price,447.669,483.461,492.491,512.654,537.541,549.745,642.491


In [21]:
round([weekly_avg_rec.Daily_Average_Price_1[1],weekly_avg_rec.Daily_Average_Price_2[1]],2)

TypeError: type list doesn't define __round__ method

In [ ]:
def create_table_week():

	part_df = partitioning(df2,df2.date[0],df2.date[len(df2.date)-1])

	datap1 = part_df.groupby('dayofyear#').apply(lambda x: x.sort_values('Total',ascending=True)).reset_index(drop = True)

	index_list = []

	for i in datap1.index:
	    if i%24 in [0,1,2,3]:# 24 will change depending on the values slected
	        index_list.append(i)

	datap2 = datap1.loc[index_list].reset_index(drop = True)

	price = pd.DataFrame(np.reshape(list(datap2.Total),(7,4)),columns=['top1_hour_price','top2_hour_price',\
                                                               'top3_hour_price','top4_hour_price'])
	date = pd.DataFrame(np.reshape(list(datap2.date),(7,4)),columns=['top1_hour_date','top2_hour_date',\
	                                                               'top3_hour_date','top4_hour_date'])
	final_dataset = pd.concat([date, price], axis = 1)
	final_dataset = final_dataset[['top1_hour_date', 'top1_hour_price','top2_hour_date', 'top2_hour_price',\
	                               'top3_hour_date', 'top3_hour_price','top4_hour_date', 'top4_hour_price']]
	final_dataset.insert(0,'date',[d.date() for d in final_dataset['top1_hour_date']])
	final_dataset.insert(1,'top1_time',[d.time() for d in final_dataset['top1_hour_date']])
	final_dataset.insert(4,'top2_time',[d.time() for d in final_dataset['top2_hour_date']])
	final_dataset.insert(7,'top3_time',[d.time() for d in final_dataset['top2_hour_date']])
	final_dataset.insert(10,'top4_time',[d.time() for d in final_dataset['top2_hour_date']])
	del final_dataset['top1_hour_date']
	del final_dataset['top2_hour_date']
	del final_dataset['top3_hour_date']
	del final_dataset['top4_hour_date']

	weekly_avg_rec = part_df.groupby('dayofyear#').mean().sort_values(by ='Total').reset_index(drop = True)
	weekly_dict = {0.0:'Monday',1.0:'Tuesday',2.0:'Wednesday',3.0:'Thursday',4.0:'Friday',5.0:'Saturday',6.0:'Sunday'}

	weekly_list = [weekly_dict[weekly_avg_rec['dayofweek#'][i]] for i in range(len(weekly_dict))]
	weekly_avg_rec.insert(5, "Week_day", weekly_list, True)
	weekly_avg_rec['Avg_price'] = weekly_avg_rec['Total']
	weekly_avg_rec = weekly_avg_rec[['Week_day','Avg_price']]
	weekly_avg_rec = weekly_avg_rec.T 
	weekly_avg_rec.columns = ['Daily_Average_Price_1','Daily_Average_Price_2','Daily_Average_Price_3',\
	                          'Daily_Average_Price_4','Daily_Average_Price_5','Daily_Average_Price_6',\
	                          'Daily_Average_Price_7']

	fig_t = go.Figure()

	date_col = '#74a7d2'
	av_col = '#b7d2eB'
	col = 'LightSkyBlue'

	fig_t.add_trace(go.Table(
	    header=dict(values=[weekly_avg_rec.Week_day[0],weekly_avg_rec.Week_day[1],weekly_avg_rec.Week_day[2],weekly_avg_rec.Week_day[3],
	    					weekly_avg_rec.Week_day[4],weekly_avg_rec.Week_day[5],weekly_avg_rec.Week_day[6]],
	                fill_color='blue',
					font=dict(color='white'),
					align='center',
					height = 30),
	    cells=dict(values=[weekly_avg_rec.Avg_price[0],weekly_avg_rec.Avg_price[1],weekly_avg_rec.Avg_price[2],weekly_avg_rec.Avg_price[3],
	    					weekly_avg_rec.Avg_price[4],weekly_avg_rec.Avg_price[5],weekly_avg_rec.Avg_price[6]],
	               fill_color=[col,'white',col,'white',col,'white',col],
	               align='left')))

	fig_t.update_layout(
		    autosize=False,
		    width=1000,
		    height=250,
		    margin=dict(
		        l=10,
		        r=10,
		        b=30,
		        t=20,
		        pad=4
		    ),
		    plot_bgcolor = '#dde9f4',
		    paper_bgcolor = '#eff5fa')

	tableJSON = json.dumps(fig_t, cls=plotly.utils.PlotlyJSONEncoder)

	return tableJSON